<a href="https://colab.research.google.com/github/HughYau/Research-with-Python/blob/main/Green%20Multiplier/GM_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import weibull_min
from interpolation import interp
from quantecon.optimize.scalar_maximization import brent_max
from collections import Counter
from matplotlib.ticker import MaxNLocator
from numba import njit, jit, njit, prange, float64, int32, typeof
from numba.experimental import jitclass
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [ ]:
stock = np.array([10933.09,12670.14,14598.11,16284.45,18574.54,20906.67,23231.22,25376.38,273409.175,301510])#2012
evstock = np.array([0,6,21.64,58.32,91.28,153.4,260.78,380.87,492.02,784])#2012
agepdf = np.array([[0.201 , 0.18  , 0.169 , 0.16  , 0.164 , 0.1466, 0.1372, 0.1278,
    0.1184, 0.109 ],
   [0.168 , 0.161 , 0.152 , 0.145 , 0.137 , 0.1292, 0.1214, 0.1136,
    0.1058, 0.098 ],
   [0.172 , 0.14  , 0.135 , 0.129 , 0.125 , 0.1087, 0.0982, 0.0877,
    0.0772, 0.0667],
   [0.129 , 0.14  , 0.117 , 0.115 , 0.109 , 0.1025, 0.096 , 0.0895,
    0.083 , 0.0765],
   [0.081 , 0.11  , 0.118 , 0.1   , 0.098 , 0.1086, 0.111 , 0.1134,
    0.1158, 0.1182],
   [0.069 , 0.066 , 0.092 , 0.099 , 0.083 , 0.1001, 0.1062, 0.1123,
    0.1184, 0.1245],
   [0.058 , 0.059 , 0.055 , 0.077 , 0.084 , 0.0876, 0.0946, 0.1016,
    0.1086, 0.1156],
   [0.042 , 0.047 , 0.048 , 0.046 , 0.064 , 0.0623, 0.0666, 0.0709,
    0.0752, 0.0795],
   [0.03  , 0.034 , 0.038 , 0.04  , 0.037 , 0.0418, 0.0438, 0.0458,
    0.0478, 0.0498],
   [0.024 , 0.024 , 0.027 , 0.031 , 0.032 , 0.0345, 0.0368, 0.0391,
    0.0414, 0.0437],
   [0.013 , 0.019 , 0.019 , 0.022 , 0.024 , 0.0269, 0.0294, 0.0319,
    0.0344, 0.0369],
   [0.006 , 0.011 , 0.015 , 0.015 , 0.017 , 0.0206, 0.0232, 0.0258,
    0.0284, 0.031 ],
   [0.004 , 0.005 , 0.008 , 0.012 , 0.011 , 0.0143, 0.0164, 0.0185,
    0.0206, 0.0227],
   [0.002 , 0.003 , 0.004 , 0.006 , 0.008 , 0.0091, 0.0106, 0.0121,
    0.0136, 0.0151],
   [0.001 , 0.001 , 0.002 , 0.003 , 0.004 , 0.0046, 0.0054, 0.0062,
    0.007 , 0.0078],
   [0.    , 0.    , 0.001 , 0.001 , 0.003 , 0.0026, 0.0032, 0.0038,
    0.0044, 0.005 ]])
truesale = np.array([[1.453,0.347],#2013
    [4.5048,2.9952],
    [24.7482,8.3518],
    [40.9,9.8],
    [65.2,12.5],
    [98.4,27.2],
    [97.2,23.4],
    [111.5,25.2]])

In [ ]:
ddcm_para = [('scene',int32),
             ('t',int32),
             ('alpha',float64),
             ('beta',float64),
             ('delta',float64),
             ('gamma',float64),
             ('A',int32),
             ('var1',float64),
             ('var2',float64),
             ('truelambda',float64),
             ('zeta',float64),
             ('epsilon',float64),
             ('AFE',float64[:]),
             ('EAFE',float64[:]),
             ('VMT',float64),
             ('y',float64),
             ('r',float64),
             ('bt',float64),
             ('vat',float64),
             ('sub',float64[:]),
             ('cha',float64),
             ('shocks',float64[:,::1]),
             ('transition',float64[:,::1])
]

@jitclass(ddcm_para)
class DDCM:
    def __init__(self,
                 scene = 0,
                 t = 7,
                 epsilon = 1,
                 alpha = 2,
                 beta=0.8, 
                 delta = 0.02, 
                 gamma = 0.34,
                 A = 20,
                 var1 = 0.2,
                 var2 = 0.2,
                 truelambda = 0.2,
                 zeta = 1.7,
                 r = 0.05,
                 shocks = np.array([[105761.01203155,   7515.46061925],
                           [104044.59375342,   8558.00847766],
                           [102356.03159778,   9745.17901352],
                           [109349.77977224,   7582.17502758],
                           [107575.11860831,   8633.97753677],
                           [105829.25880323,   9831.68653247],
                           [113060.32446693,   7649.48165673],
                           [111225.44407274,   8710.62096983],
                           [109420.34234828,   9918.96197481]]),
                 transition = np.array([[1.97170195e-01, 4.46763023e-01, 1.38531452e-01, 2.89016362e-02,
                                6.54874961e-02, 2.03062417e-02, 2.59142361e-02, 5.87184208e-02,
                                1.82072994e-02],
                               [7.40998999e-02, 3.42063659e-01, 3.65482201e-01, 1.09625370e-02,
                                5.06058109e-02, 5.40704127e-02, 9.73741194e-03, 4.49503273e-02,
                                4.80277403e-02],
                               [2.66343353e-03, 1.34830014e-01, 6.43333900e-01, 3.97663163e-04,
                                2.01307558e-02, 9.60527797e-02, 3.49943580e-04, 1.77150649e-02,
                                8.45264452e-02],
                               [1.13618061e-01, 7.97603087e-02, 1.86765138e-02, 3.08931475e-01,
                                2.16871064e-01, 5.07820931e-02, 1.13246005e-01, 7.94991241e-02,
                                1.86153553e-02],
                               [4.48200448e-02, 1.22067362e-01, 4.48200458e-02, 1.22067361e-01,
                                3.32450377e-01, 1.22067364e-01, 4.48200431e-02, 1.22067358e-01,
                                4.48200442e-02],
                               [1.86153553e-02, 7.94991211e-02, 1.13246002e-01, 5.07820943e-02,
                                2.16871061e-01, 3.08931473e-01, 1.86765152e-02, 7.97603119e-02,
                                1.13618066e-01],
                               [8.45264525e-02, 1.77150659e-02, 3.49943590e-04, 9.60527754e-02,
                                2.01307543e-02, 3.97663123e-04, 6.43333902e-01, 1.34830010e-01,
                                2.66343338e-03],
                               [4.80277339e-02, 4.49503235e-02, 9.73741061e-03, 5.40704161e-02,
                                5.06058167e-02, 1.09625377e-02, 3.65482194e-01, 3.42063669e-01,
                                7.40998982e-02],
                               [1.82073008e-02, 5.87184255e-02, 2.59142379e-02, 2.03062428e-02,
                                6.54874999e-02, 2.89016376e-02, 1.38531450e-01, 4.46763016e-01,
                                1.97170189e-01]])
             ):

        self.t,self.epsilon,self.delta, self.beta, self.gamma,self.alpha,self.A,self.var1,self.var2,self.truelambda,self.zeta,self.r = t,epsilon,delta, beta, gamma,alpha,A,var1,var2,truelambda,zeta,r
        self.scene = scene
        self.shocks = shocks
        self.transition = transition
        

        
        INC = np.array([18331,20167,21966,23821,25974,28228,30733,32189,35128])
        self.AFE = np.array([6.79,6.99,7.22,7.33,7.38,7.54,7.71,7.77,7.89,8,8.06,8.65,9.15,9.65,10.15,10.65,11.15,11.65,12.15,12.65,13.15])
        self.EAFE = np.array([14.7,19])
        self.VMT = 150
        N = np.array([2.98,2.97,3.1,3.11,3.03,3,2.92,2.92,3])
        VAT = np.array([0.17,0.17,0.17,0.17,0.17,0.16,0.13,0.13,0.13,0.13,0.13])
        if scene == 0:
            SUB = np.array([[0,4.83,3.5],
                     [0,4.59,3.325],
                     [0,4.35,3.15],
                     [0,4.17,3],
                     [0,3.33,2.4],
                     [0,3.36,2.2],
                     [0,2.15,1],
                     [0,1.94,0.85],
                     [0,1.55,0.68],
                     [0,1.08,0.476]])
            self.bt = 0.1
        elif scene == 1:
            self.bt  = 0.1
            SUB = np.array([[0,4.83,3.5],
                     [0,4.83,3.5],
                     [0,4.83,3.5],
                     [0,4.83,3.5],
                     [0,4.83,3.5],
                     [0,4.83,3.5],
                     [0,4.83,3.5],
                     [0,4.83,3.5],
                     [0,4.83,3.5],
                     [0,4.83,3.5]])
        elif scene == 2:
            self.bt = 0.1
            SUB = np.array([[0,4.83,3.5],
                         [0,4.59,3.325],
                         [0,4.35,3.15],
                         [0,4.17,3],
                         [0,3.33,2.4],
                         [0,3.36,2.2],
                         [0,3.36,2.2],
                        [0,3.36,2.2],
                         [0,3.36,2.2],
                         [0,3.36,2.2]])
        elif scene == 3:
            SUB = np.array([[0,4.83,3.5],
                     [0,4.59,3.325],
                     [0,4.35,3.15],
                     [0,4.17,3],
                     [0,3.33,2.4],
                     [0,3.36,2.2],
                     [0,2.15,1],
                     [0,1.94,0.85],
                     [0,1.55,0.68],
                     [0,1.08,0.476]])
            self.bt = 0
        elif scene == 4:
            SUB = np.zeros((10,3))
            self.bt = 0
        elif scene == 5:
            SUB = np.zeros((10,3))
            self.bt = 0.1
        CHARGE = np.array([2.2528,3.0914,6.6,23,44.6,77.7,121.9,168.1,261.7])
        
        self.y = self.epsilon*N[self.t]*INC[self.t]+80000
        self.vat = VAT[self.t]
        self.sub = SUB[self.t]
        self.cha = CHARGE[self.t]
        
    def F(self,i,a,e):
        O = e[1]/1000
        if i == 0:
            F = 0.09*self.y
        elif i == 1:
            F = self.AFE[a-1]*self.VMT*0.732*O
        elif i == 2:
            F = self.EAFE[i-2]*self.VMT*0.6
        else:
            F = (self.EAFE[i-2]*0.6+self.AFE[a-1]*0.732*O)*self.VMT/2

        return F


    def constraint(self,c,i,a,e):
        P0 = e[0]
        def Q(i,a):
            scrapdis = np.array([0.806225  , 0.756255  , 0.695555  , 0.62868824, 0.5709375 ])
            scrapEV = np.array([0.612853333 ,0.62882])
            if i == 1:
                if a < 6:
                    Q = P0*scrapdis[i]
                else:
                    Q = 10000
            elif i == 2 or i == 3:
                if a == 1:
                    Q = P0*scrapEV[i-2]
                else:
                    Q = 0
            else:
                Q = 0

            return Q
        if c == 0:
            con = self.y - self.F(i,a,e)
        elif c == 4:
            con = self.y - self.F(0,a,e)+Q(i,a)
        else:
            P = P0*(1+self.bt+self.vat)/(1+self.vat)-self.sub[c-1]*10000
            con = self.y - self.F(c,1,e)-P+Q(i,a)

        return con

    def utility(self,c,i,a,e):
        aveage = np.array([10.5,10.5,12.1,12.1,12.9,12.9,12.9,12.9,12.9])
        cha = np.array([0,self.cha**self.var1,self.cha**self.var2])
        C = self.constraint(c,i,a,e)
        if C < 0:
            C = 10
        if c == 0:
            if i == 0:
                u = (C/self.truelambda)**(1-self.zeta)/(1-self.zeta)+self.alpha
            else:
#                 u = 1/(1+np.exp(self.gamma*(a-aveage[self.t])))+(C/self.truelambda)**(1-self.zeta)/(1-self.zeta)
                u = a**(-self.gamma)+(C/self.truelambda)**(1-self.zeta)/(1-self.zeta)+self.alpha
        elif c == 4:
            u = (C/self.truelambda)**(1-self.zeta)/(1-self.zeta)

        else:
            u = 1+(C/self.truelambda)**(1-self.zeta)/(1-self.zeta)+cha[c-1]

        return u

@jit(nopython = True)
def T(w,ddcm):
        Tw = np.zeros((4,ddcm.A,len(ddcm.shocks)))
        flag = np.asarray(Tw)
        for i in range(4):
            for k,e in enumerate(ddcm.shocks):
                if i == 0:
                    keep = ddcm.utility(0,0,0,e) + ddcm.beta*np.dot(ddcm.transition[k],w[0][0])
                    replace1 = ddcm.utility(1,0,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[1][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                    replace2 = ddcm.utility(2,0,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[2][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                    replace3 = ddcm.utility(3,0,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[3][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                    Tw[0][0][k] = max(keep,replace1,replace2,replace3)


                else:
                    for j in range(1,ddcm.A-1):
                        if i == 1:
                            keep = ddcm.utility(0,1,j,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[i][j+1])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                            replace1 = ddcm.utility(1,1,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[1][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                            replace2 = ddcm.utility(2,1,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[2][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                            replace3 = ddcm.utility(3,1,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[3][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                            scrap = ddcm.utility(4,1,0,e)+ ddcm.beta*np.dot(ddcm.transition[k],w[0][0])
                            
                            Tw[i][j][k] = max(keep,replace1,replace2,replace3,scrap)

                        else:
                            keep = ddcm.utility(0,i,j,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[i][j+1])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                            scrap = ddcm.utility(4,i,0,e)+ ddcm.beta*np.dot(ddcm.transition[k],w[0][0])
                            
                            Tw[i][j][k] = max(keep,scrap)
                            

        return Tw
    
    
@jit(nopython = True)
def Prob(ddcm,w):
    Prk = np.zeros((4,ddcm.A,len(ddcm.shocks)))
    Prr1 = np.zeros((4,ddcm.A,len(ddcm.shocks)))
    Prr2 = np.zeros((4,ddcm.A,len(ddcm.shocks)))
    Prr3 = np.zeros((4,ddcm.A,len(ddcm.shocks)))
    Prs = np.zeros((4,ddcm.A,len(ddcm.shocks)))

    for i in range(4):
        for k,e in enumerate(ddcm.shocks):
            if i == 0:
                keep = ddcm.utility(0,0,0,e) + ddcm.beta*np.dot(ddcm.transition[k],w[0][0])
                replace1 = ddcm.utility(1,0,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[1][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                replace2 = ddcm.utility(2,0,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[2][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                replace3 = ddcm.utility(3,0,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[3][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))

                Prk[0][0][k] = np.exp(keep)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3))
                Prr1[0][0][k] = np.exp(replace1)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3))
                Prr2[0][0][k] = np.exp(replace2)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3))
                Prr3[0][0][k] = np.exp(replace3)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3))


            else:
                for j in range(1,ddcm.A-1):
                    if i == 1:
                        keep = ddcm.utility(0,1,j,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[i][j+1])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                        replace1 = ddcm.utility(1,1,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[1][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                        replace2 = ddcm.utility(2,1,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[2][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                        replace3 = ddcm.utility(3,1,1,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[3][2])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                        scrap = ddcm.utility(4,1,0,e)+ ddcm.beta*np.dot(ddcm.transition[k],w[0][0])

                        Prk[1][j][k] = np.exp(keep)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3)+np.exp(scrap))
                        Prr1[1][j][k] = np.exp(replace1)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3)+np.exp(scrap))
                        Prr2[1][j][k] = np.exp(replace2)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3)+np.exp(scrap))
                        Prr3[1][j][k] = np.exp(replace3)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3)+np.exp(scrap))
                        Prs[1][j][k] = np.exp(scrap)/(np.exp(keep)+np.exp(replace1)+np.exp(replace2)+np.exp(replace3)+np.exp(scrap))

                    else:
                        keep = ddcm.utility(0,i,j,e) + ddcm.beta*((1-ddcm.delta)*np.dot(ddcm.transition[k],w[i][j+1])+ddcm.delta*np.dot(ddcm.transition[k],w[0][0]))
                        scrap = ddcm.utility(4,i,0,e)+ ddcm.beta*np.dot(ddcm.transition[k],w[0][0])

                        Prk[i][j][k] = np.exp(keep)/(np.exp(keep)+np.exp(scrap))
                        Prs[i][j][k] = np.exp(scrap)/(np.exp(keep)+np.exp(scrap))


    return  Prk,Prr1 ,Prr2,Prr3 , Prs 


def solve_model(v,ddcm ,
                tol=1e-4,
                max_iter=1000,
                verbose=False,
                print_skip=1):


    i = 0
    error = tol + 1

    while i < max_iter and error > tol:
        v_new = T(v,ddcm)
        error = np.max(np.abs(v - v_new))
        i += 1
        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")
        v = v_new

    if i == max_iter:
        print("Failed to converge!")

    if verbose and i < max_iter:
        print(f"\nConverged in {i} iterations.")

    return v_new



In [ ]:
print(truesale)
ep = np.array([0.12963452764342645,
0.39477314210369746,
0.722465667274827,
1.1993295554449177,
2.376092686067955])
theta0 = [7.33086700e+00, 1.17610371e+01, 1.95900904e-01, 6.91557514e-02,
       8.72723806e+05, 2.13174732e+00]
def estimate_function(theta,shocklist,s):
    trstock = [stock[i] - evstock[i] for i in range(len(stock))]
    year = 8
#     shocklist=[5,5,1,0,1,5,7,6,8]
    predsale1 = np.empty((year,2))
    predsale2 = np.empty((year,2))
    theta1 = [2.13407917e+01, 2.32133644e-01, 1.67305407e-01, 1.17373928e-01, 2.13642540e+05,
 2.40311742e+00]
    for i in range(0,year):
        Pr = 0
        for e_i,e in enumerate(ep):
            k = shocklist[i]
            model = DDCM(scene = s,t = i,epsilon = e,alpha = theta[0], beta=0.95, delta = 0.02, gamma = theta[1],A = 16,var1 = theta[2],var2 = theta[3],truelambda = theta[4],zeta = theta[5])
            w_guess = np.zeros((4,model.A,len(model.shocks)))
            w_star1  = solve_model(w_guess,model, print_skip=100)
#         Pr = Prob(model,w_star1)
            Pr = Pr+np.asarray(Prob(model,w_star1))
        Pr = Pr/len(ep)
        for j in range (0,2):
            frac1 = 0
            frac2 = 0
            for a in range(1,15):
                frac1 += Pr[j+2][1][a+1][k]*(1-model.delta)*agepdf[a][i]
                frac2 += Pr[4][1][a+1][k]*agepdf[a][i]
            predsale1[i][j] = frac1*trstock[i]
            predsale2[i][j] = Pr[j+2][0][0][k]*trstock[i]*(model.delta+frac1+frac2)
#     print(predsale)
    return predsale1,predsale2
# for i in range(6):
#     print("第",i)
def estimate_function_wealth(theta,shocklist,s):
    trstock = [stock[i] - evstock[i] for i in range(len(stock))]
    year = 8
#     shocklist=[5,5,1,0,1,5,7,6,8]
    predsale = np.empty((5,year,2))
    theta1 = [2.13407917e+01, 2.32133644e-01, 1.67305407e-01, 1.17373928e-01, 2.13642540e+05,
 2.40311742e+00]
    for i in range(0,year):
        Pr = 0
        for e_i,e in enumerate(ep):
            k = shocklist[i]
            model = DDCM(scene = s,t = i,epsilon = e,alpha = theta[0], beta=0.95, delta = 0.02, gamma = theta[1],A = 16,var1 = theta[2],var2 = theta[3],truelambda = theta[4],zeta = theta[5])
            w_guess = np.zeros((4,model.A,len(model.shocks)))
            w_star1 = w_inc = solve_model(w_guess,model, print_skip=100)
#         Pr = Prob(model,w_star1)
            Pr = np.asarray(Prob(model,w_star1))/len(ep)
            for j in range (0,2):
                frac1 = 0
                frac2 = 0
                for a in range(1,15):
                    frac1 += Pr[j+2][1][a+1][k]*(1-model.delta)*agepdf[a][i]
                    frac2 += Pr[4][1][a+1][k]*agepdf[a][i]
                predsale[e_i][i][j] = (frac1*trstock[i]+Pr[j+2][0][0][k]*trstock[i]*(model.delta+frac1+frac2))
#     print(predsale)
    return predsale

predsale1 = np.empty((6,8,2))
predsale2 = np.empty((6,8,2))
predsale_wealth = np.empty((6,5,8,2))
predsale_pd = pd.DataFrame()
predsale_wealth_pd = pd.DataFrame()
for i in range(6):
    pred = estimate_function([2.07139266e+01, 5.99218827e-01, 1.87930275e-01, 1.47336089e-01,
       1.11396624e+06, 1.71926020e+00],[6,6,1,4,2,2,4,6],i)
#     predsale_pd = pd.concat([predsale_pd,pd.DataFrame(pred)],axis = 1)
    predsale1[i] = pred[0]
    predsale2[i] = pred[1]
    
# for i in range(6):
#     pred = estimate_function_wealth([2.07139266e+01, 5.99218827e-01, 1.87930275e-01, 1.47336089e-01,
#        1.11396624e+06, 1.71926020e+00],[6,6,1,4,2,2,4,6],i)
#     predsale_wealth_pd = pd.concat([predsale_wealth_pd,pd.DataFrame(pred.T.reshape(-1,5))],axis = 1)
#     predsale_wealth[i] = pred

[[  1.453    0.347 ]
 [  4.5048   2.9952]
 [ 24.7482   8.3518]
 [ 40.9      9.8   ]
 [ 65.2     12.5   ]
 [ 98.4     27.2   ]
 [ 97.2     23.4   ]
 [111.5     25.2   ]]


In [ ]:
# np.save('predsale_new.npy',predsale)
# np.save('predsale_wealth_new.npy',predsale_wealth)
np.save('predsale1_new.npy',predsale1)
np.save('predsale2_new.npy',predsale2)

## Sensitive Analysis

In [ ]:
predsale1 = np.load('predsale1_new.npy')
predsale2 = np.load('predsale2_new.npy')

In [ ]:
predsale1

array([[[  6.68852399,   1.48413222],
        [ 11.00918469,   2.64278198],
        [ 23.00780989,   5.34427639],
        [ 43.29940463,  10.04095534],
        [ 62.18886901,  14.57981211],
        [ 98.32913122,  20.93208056],
        [ 94.78750549,  22.64788869],
        [113.87303473,  28.69108929]],

       [[  6.68852399,   1.48413222],
        [ 11.88039868,   2.76655807],
        [ 26.45417254,   5.79165715],
        [ 51.72229312,  11.13949384],
        [ 91.3263012 ,  17.85639047],
        [139.59492458,  26.54536977],
        [181.38814425,  35.5548348 ],
        [222.12742156,  44.80247454]],

       [[  6.68852399,   1.48413222],
        [ 11.00918469,   2.64278198],
        [ 23.00780989,   5.34427639],
        [ 43.29940463,  10.04095534],
        [ 62.18886901,  14.57981211],
        [ 98.32913122,  20.93208056],
        [130.56754683,  28.55411078],
        [163.56956238,  36.72537114]],

       [[  8.97547838,   1.9393896 ],
        [ 14.7045283 ,   3.4081903 ],
      

In [ ]:
#https://auto.ifeng.com/c/8AWDJtdbt0E
temp1 = predsale1[:,:,1]+predsale1[:,:,0]
temp2 = (predsale2[:,:,1]+predsale2[:,:,0])*10e7

In [ ]:
predsale1.T

array([[[  6.68852399,   6.68852399,   6.68852399,   8.97547838,
           1.4797996 ,   0.9955637 ],
        [ 11.00918469,  11.88039868,  11.00918469,  14.7045283 ,
           2.8545291 ,   1.97331597],
        [ 23.00780989,  26.45417254,  23.00780989,  29.49394034,
           6.89831537,   5.01698746],
        [ 43.29940463,  51.72229312,  43.29940463,  55.02714626,
          15.14473956,  11.2052316 ],
        [ 62.18886901,  91.3263012 ,  62.18886901,  78.27982723,
          28.78963266,  21.85929072],
        [ 98.32913122, 139.59492458,  98.32913122, 122.18148366,
          47.12517452,  36.26712534],
        [ 94.78750549, 181.38814425, 130.56754683, 122.16247629,
          66.58287932,  51.23557784],
        [113.87303473, 222.12742156, 163.56956238, 147.28406749,
          87.28593983,  67.42077728]],

       [[  1.48413222,   1.48413222,   1.48413222,   1.9393896 ,
           0.68636445,   0.4983723 ],
        [  2.64278198,   2.76655807,   2.64278198,   3.4081903 ,
      

In [ ]:
c = np.zeros((6,8))
c_prim = np.zeros((6,8))
cg1 = 55.3
cg2 = 28.1 

for i in range(6):
    c[i] = (predsale1[i,:,0]*cg1+predsale1[i,:,1]*cg2-(predsale2[i,:,0]*10e7+predsale2[i,:,1]*10e7)*0.5*209)*15/100
    c_prim[i]  = (predsale1[i,:,0]*cg1+predsale1[i,:,1]*cg2)*15/100
#     print(c)

In [ ]:
c-c[5]

array([[  38.38107715,   64.38876599,  123.46825359,  230.22736884,
         285.41676702,  431.0082065 ,  307.89322367,  331.17321533],
       [  38.38107715,   70.10058646,  144.08342394,  284.28344328,
         461.73271521,  673.58729619,  868.76979857, 1055.52623559],
       [  38.38107715,   64.38876599,  123.46825359,  230.22736884,
         285.41676702,  431.0082065 ,  558.27650117,  686.47875224],
       [  51.00472797,   87.78328358,  160.96460371,  304.88077619,
         388.81314559,  579.41889763,  500.78701619,  572.92358526],
       [   3.632358  ,    6.76150489,   13.85901175,   30.2272493 ,
          51.91586832,   80.5654671 ,  115.14373333,  150.97912851],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ]])

In [ ]:
predsale2*10e7

array([[[ 0.69672526,  0.21261842],
        [ 0.96842508,  0.32906909],
        [ 2.3697416 ,  0.79785665],
        [ 3.81036723,  1.24702716],
        [ 5.75353025,  1.89610978],
        [ 9.54853898,  2.75077253],
        [ 8.44935617,  2.64225497],
        [ 9.74127403,  3.10185411]],

       [[ 0.69672526,  0.21261842],
        [ 1.07476082,  0.35266099],
        [ 2.90100631,  0.89549896],
        [ 4.91606346,  1.44545375],
        [10.14640079,  2.55522199],
        [16.33906285,  3.83142443],
        [19.88653187,  4.72204906],
        [22.77172491,  5.47979924]],

       [[ 0.69672526,  0.21261842],
        [ 0.96842508,  0.32906909],
        [ 2.3697416 ,  0.79785665],
        [ 3.81036723,  1.24702716],
        [ 5.75353025,  1.89610978],
        [ 9.54853898,  2.75077253],
        [12.15502071,  3.48567441],
        [14.48740483,  4.1478546 ]],

       [[ 1.11085352,  0.3257977 ],
        [ 1.48553415,  0.48083106],
        [ 3.46943243,  1.07508245],
        [ 5.45042299, 

In [ ]:
ep = np.array([0.12963452764342645,
0.39477314210369746,
0.722465667274827,
1.1993295554449177,
2.376092686067955])
theta = [2.07139266e+01, 5.99218827e-01, 1.87930275e-01, 1.47336089e-01,
       1.11396624e+06, 1.71926020e+00]
model = DDCM(scene = 0,t = 7,epsilon = 1.1993295554449177,alpha = theta[0], beta=0.95, delta = 0.02, gamma = theta[1],A = 16,var1 = theta[2],var2 = theta[3],truelambda = theta[4],zeta = theta[5])
w_guess = np.zeros((4,model.A,len(model.shocks)))
w_star1 = w_inc = solve_model(w_guess,model, print_skip=100)
Pr = Prob(model,w_star1)

In [ ]:
Pr[0][3]